In [1]:
# 1. Introduction to Kaggle
# --- Torch ---
import numpy as np

test_t  = np.genfromtxt('KaggleData/kaggle_test.csv', delimiter=",")
print(test_t.shape)

train_t = np.genfromtxt('KaggleData/kaggle_train.csv', delimiter=",")
print(train_t.shape)

In [2]:
# --- Kaggle ---
import pandas as pd

test = pd.read_csv("KaggleData/test.csv")
test_shape = test.shape
print(test_shape)

train = pd.read_csv("KaggleData/train.csv")
train_shape = train.shape
print(train_shape)

(418, 11)
(891, 12)


In [3]:
train.head(2)


In [4]:

def process_age(df,cut_points,label_names):
    df["Age"] = df["Age"].fillna(-0.5)
    df["Age_categories"] = pd.cut(df["Age"],cut_points,labels=label_names)
    return df

cut_points = [-1,0, 5, 12, 18, 35, 60, 100]
label_names = ["Missing", 'Infant', "Child", 'Teenager', "Young Adult", 'Adult', 'Senior']

train = process_age(train,cut_points,label_names)
test = process_age(test,cut_points,label_names)

In [5]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

def prepare_data(train, test):
    train = create_dummies(train,"Pclass")
    train = create_dummies(train,"Sex")
    train = create_dummies(train,"Age_categories")
    
    test = create_dummies(test,"Pclass")
    test = create_dummies(test,"Sex")
    test = create_dummies(test,"Age_categories")
    
    return train, test

In [6]:
# 2. Exploring the data

columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior']

In [7]:
# --- Kaggle ---
train, test = prepare_data(train, test)
train.head(2)  # TODO why it doesn't work?

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Pclass_3,Sex_female,Sex_male,Age_categories_Missing,Age_categories_Infant,Age_categories_Child,Age_categories_Teenager,Age_categories_Young Adult,Age_categories_Adult,Age_categories_Senior
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,1,0,1,0,0,0,0,1,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,0,1,0,0,0,0,0,0,1,0


KeyError: 1

In [13]:

# *********** Load the data **********


KeyError: 1

In [14]:

train.head(2)


KeyError: 1

In [15]:
# Convert Pandas dataframe to PyTorch tensor
import random

# creating dummy targets (float values)
targets_data = [random.random() for i in range(train[1])]
print('targets_data:', targets_data)

# creating DataFrame from targets_data
targets_df = pd.DataFrame(data=targets_data)
targets_df.columns = columns

# creating tensor from targets_df 
torch_tensor = torch.tensor(targets_df['targets'].values)

# printing out result
print(torch_tensor)

KeyError: 1

In [11]:
# *********** Create the model **********
from torch import nn
from collections import OrderedDict

model = nn.Sequential(OrderedDict([
    ('fc1',  nn.Linear(train.shape[0], 512)),
    ('relu', nn.ReLU()),
    ('drop', nn.Dropout(p=0.2)),
    ('fc2',  nn.Linear(512, 128)),
    ('relu', nn.ReLU()),
    ('fc3',  nn.Linear(128, 2)),
    ('output', nn.LogSoftmax(dim=1))
]))

In [12]:
# *********** Train the model **********
import torch as th
import torch.utils.data

num_teachers = 100
num_examples = train.shape[0]
num_labels = 2


device = th.device('cuda:0' if th.cuda.is_available() else 'cpu')

criterion = nn.NLLLoss()
optimizer = th.optim.Adam(model.parameters(), lr=0.001)
   
model.to(device)

epochs = 5
step = 0

preds = []

for i in range(num_teachers):
    print('Trainset:', i+1)
    pred = []

    for e in range(epochs):
        #print('Epoch:', e+1)
        running_loss = 0
        trainloader = th.utils.data.DataLoader(train_t[i], batch_size=1, shuffle=True)
        print('trainloader:', trainloader.shape)
        
        for imgs, labels in trainloader:
            step += 1
            imgs = imgs.view(imgs.shape[0], -1)
            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()

            logps = model(imgs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
    else:
        with th.no_grad():
            model.eval()
            test_loss = 0
            accuracy  = 0

            for imgs, labels in testloader:
                imgs = imgs.view(imgs.shape[0], -1)
                imgs, labels = imgs.to(device), labels.to(device)

                logps = model(imgs)
                loss = criterion(logps, labels)
                test_loss += loss.item()

                ps = th.exp(logps)
                top_ps, top_class = ps.topk(1, dim=1)
                equality = top_class == labels.view(*top_class.shape)
                accuracy += th.mean(equality.type(th.FloatTensor)).item()

                pred.append(top_class.cpu().flatten())
        model.train()

        print( 'Training loss:', running_loss/(epochs*num_teachers) )
        print( 'Test loss:', test_loss/len(testloader) )
        print( 'Accuracy:', accuracy/len(testloader) )

    preds.append( np.array(pred) )
    print('Pred shape:', np.array(pred).shape)
preds = np.array(preds)
print('Preds shape:', preds.shape)

Trainset: 1


NameError: name 'train_t' is not defined